In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai import *
from fastai.vision import *
import os
import pandas as pd
# from sklearn.model_selection import train_test_split

In [ ]:
print(os.listdir("../input/vietaiass3/VietAI-3"))
data_dir = '../input/vietaiass3/VietAI-3'

In [ ]:
new_path_train = '../input/trainnewcsv'
train_df = pd.read_csv(os.path.join(new_path_train, 'new_train.csv'))
# train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))

train_df.head()

In [ ]:
train_df.label.value_counts()

In [ ]:
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
test_df.head()

In [ ]:
data = ImageDataBunch.from_csv(
    path = '../input/',
    folder= 'vietaiass3/VietAI-3/images',
    csv_labels = 'trainnewcsv/new_train.csv',
#     csv_labels='vietaiass3/VietAI-3/train.csv',
    test = 'vietaiass3/VietAI-3/images/test',
    valid_pct = 0.2,
    bs = 32,
    size = 350,
    ds_tfms = get_transforms(),
    num_workers = 0
).normalize(imagenet_stats)

In [ ]:
data

In [ ]:
print(data.classes)
data.show_batch()

In [ ]:
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp '../input/resnet152/resnet152-b121ed2d.pth' '/tmp/.cache/torch/checkpoints/resnet152-b121ed2d.pth'

In [ ]:
learn = cnn_learner(data, models.resnet152, metrics=accuracy, model_dir="/tmp/models")

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn.save('stage-1')

In [ ]:
#Initiating refit and checking LR
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,max_lr=slice(1e-06,1e-05))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.recorder.plot()

In [ ]:
# #Initiating refit and checking LR
# learn.unfreeze()
# learn.lr_find()

In [ ]:
# learn.recorder.plot()

In [ ]:
# learn.fit_one_cycle(2,max_lr=slice(1e-06,1e-05))

In [ ]:
# learn.save('stage-3')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_multi_top_losses(3, figsize=(6,6))

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
learn.load('stage-2')

In [ ]:
class_score, y = learn.get_preds(DatasetType.Test)
class_score = np.argmax(class_score, axis=1)

In [ ]:
predicted_classes = [data.classes[i] for i in class_score]
predicted_classes[:10]

In [ ]:
test_dir = '../input/vietaiass3/VietAI-3/images/test'

In [ ]:
submission  = pd.DataFrame({
    "image": os.listdir(test_dir),
    "label": predicted_classes
})
submission.to_csv("submission.csv", index=False)
submission[:10]

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
# df = pd.DataFrame(np.arange(1000), columns=['data'])

submission.to_csv("submission.csv", index=False)

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission.csv')